In [6]:
import os
import time
import pickle
import numpy as np
import multiprocessing as mp
from IPython.display import SVG

from pydrake.geometry import MeshcatVisualizer, MeshcatVisualizerParams, Role, StartMeshcat
from pydrake.geometry.optimization import IrisInConfigurationSpace, IrisOptions
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.solvers import MosekSolver
from pydrake.systems.framework import DiagramBuilder

from gcs.linear import LinearGCS
from gcs.rounding import randomForwardPathSearch
from reproduction.prm_comparison.helpers import (
    get_traj_length,
    InverseKinematics,
    make_traj,
    visualize_trajectory
)

from reproduction.util import GcsDir, FindModelFile

# mosek_license = MosekSolver.AcquireLicense()

In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant)
parser.package_map().Add("gcs", GcsDir())
# parser.package_map().GetPackageNames()
print(parser.package_map().GetPath("drake"))
print(parser.package_map().GetPath("drake_models"))
print(parser.package_map().GetPath("gcs"))

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/2c961001bc919203c70722363e3be9ea390d98aa.tar.gz


/home/lhc/opt/drake-build/install/lib/python3.10/site-packages/pydrake/common/../../../../../share/drake
/home/lhc/.cache/drake/package_map/fc028ede0b3ecbabaf8a7bb6ae5a4321bbb0b1d313f79108b9ff2b2114ac5c8c-1c1ebb431b32afef1c7f9c4cbe79bedc90438bb9e2a71a2eb92b54ac0fd07585
/home/lhc/lhc-gcs-science-robotics


In [25]:
from pydrake.geometry import SceneGraph, RoleAssign, MakePhongIllustrationProperties
from pydrake.multibody.plant import MultibodyPlant
meshcat = StartMeshcat()
meshcat.Delete()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

parser = Parser(plant, scene_graph)
parser.package_map().Add("gcs", GcsDir())

directives_file = FindModelFile(
    "models/iiwa14_spheres_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
iiwa, wsg, _, _, _, _ = ProcessModelDirectives(directives, plant, parser)

# Set transparency of main arm and gripper.
alpha = 0.7
inspector = scene_graph.model_inspector()

for body_id in plant.GetBodyIndices(iiwa.model_instance):
    frame_id = plant.GetBodyFrameIdOrThrow(body_id)
    for geometry_id in inspector.GetGeometries(frame_id,
                                                Role.kIllustration):
        properties = inspector.GetIllustrationProperties(geometry_id)
        print(properties.GetGroupNames())
        phone = MakePhongIllustrationProperties([0,0,1,0.5])
        phong = phone.GetProperty("phong", "diffuse")
        phong.set(phong.r(), phong.g(), phong.b(), alpha)
        properties.UpdateProperty("phong", "diffuse", phong)
        scene_graph.AssignRole(plant.get_source_id(), geometry_id,
                                properties, RoleAssign.kReplace)
plant.Finalize()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

INFO:drake:Meshcat listening for connections at http://localhost:7002


{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}
{'__default__'}


In [23]:
diagram = builder.Build()

context = diagram.CreateDefaultContext()


In [24]:
diagram.ForcedPublish(context)